# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2025

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^256 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^256.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 512 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.


# Support function for querying the web API


In [23]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vector entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://rasmuspagh.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API


In [24]:
from scipy.linalg import hadamard

challenge_id = 'GodOfWar5' # identifier for hidden dataset
n = 256 # number of entries in hidden dataset
assert (n & (n - 1)) == 0, "n must be a power of two for Hadamard."

H = hadamard(n).astype(int)

repeats = 2 # 2 repeats × 256 rows = 512 total
queries = np.vstack([H for _ in range(repeats)]) # shape (512, 256)
answers = query(challenge_id, queries, submit=False) # shape (512,)
answers_avg = (answers[:n] + answers[n:]) / 2.0 # shape (256,)
query_results = answers_avg
print(query_results)


[  0.   42.5  31.5 -20.   25.  -36.5  16.5  20.5  54.5  17.  -21.    3.5
  -8.  -32.  -26.   24.  -12.  -47.  -13.  -16.   46.5 -19.5 -40.   -2.
 -12.   27.5 -41.5 -47.   14.    7.5  20.5  20.  -33.5 -15.  -14.   -8.
  -4.    7.   19.  -15.   -5.   16.   15.5  16.  -24.   27.   33.   14.
 -12.  -10.5   7.5 -13.5  48.  -33.5 -10.   32.5 -21.5  11.5  32.   -5.5
 -24.  -13.   52.  -16.   13.  -26.   39.5 -22.5 -11.5   3.5 -53.5 -11.
  -2.5  24.5  -6.  -31.5 -36.5  18.5  27.5 -19.5   7.5  -9.5  -7.  -12.
  14.5  21.   -7.  -30.   -4.  -24.5  -4.5  16.5  13.   -8.  -22.  -13.
 -11.5  10.  -14.   17.  -46.5  16.5 -14.5   8.5  34.    7.  -25.   42.5
  26.5  -8.    8.  -35.   10.   35.  -19.5  -7.   -3.  -19.  -16.5 -15.5
 -16.5 -10.   45.  -41.5  -4.  -14.    5.   -1.5  11.   -1.5 -17.5  22.
 -11.5  39.    9.5  49.5   2.    8.   38.5  16.5 -40.5  14.    6.5  22.5
  27.  -14.5 -18.   36.   38.  -18.  -14.5  -8.5  16.   -9.5   8.    2.
 -15.   10.  -27.   30.5  -2.  -24.5 -19.5 -28.   -5.   18.

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://rasmuspagh.pythonanywhere.com/leaderboard/?n=256


**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```


In [25]:
y_hat = (H.T @ query_results) / n

guess = np.where(y_hat >= 0, 1, -1).astype(int)

best_query_result = query(challenge_id, guess, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.8515625 correct values
